<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>Introduction to Point Spread Function (PSF) - Part a): data products</b> <br>
Contact author(s): Andrés A. Plazas Malagón<br>
Last verified to run:  09/5/2023 <br>
LSST Science Pipelines version: w_2023_21  <br>
Container Size: medium <br>
Targeted learning level: intermediate <br>

**Description:** A demonstration on accesing `calexp` and `deepCoadd` PSF properties.

**Skills:** Use of single-epoch and coadded PSF models.

**LSST Data Products:**   DP0.2 collection: `2.2i/runs/DP0.2`. Dataset types: `calexp`, `deepCoadd`.  Catalogs: `dp02_dc2_catalogs.Object`, `dp02_dc2_catalogs.Source`.

**Packages:** lsst.afw.image, lsst.afw.detection, lsst.rsp, lsst.daf.butler, lsst.geom, lsst.afw.display, lsst.analysis.tools, lsst.pipe.base

**Credits:** Developed by Andrés A. Plazas Malagón in collaboration with Melissa Graham, Jeff Carlin, Douglas Tucker, Ryan Lau, and the Rubin Community Science Team for DP0.2. The `ImageExaminer` class, used for studying the PSF image profile and properties, is built upon the rapid-analysis code originally created by Merlin Fisher-Levine to characterize the Point Spread Function (PSF) of the Rubin Auxiliary Telescope LSST Atmospheric Transmission and Slitless Spectrograph (LATISS) images. 

This notebook incorporates suggestions from the [Accessible Authoring Checklist](https://iota-school.github.io/accessibility_hackathon/hack#checklist) and utilizes NASA's Astrophysics Data System Bibliographic Services.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

The Point Spread Function (PSF) can be understood as a function that describes how a bundle of rays, initially converging on a single point, spreads out spatially around that point. To accurately characterize the PSF, it is common to analyze the observed images of stars, which act as point sources before being distorted by the PSF. This analysis helps determine the convolution kernel, representing the size and shape of the blurring effect caused by the PSF. Accurate PSF modeling is crucial because any inaccuracies can lead to erroneous conclusions about fundamental aspects of the universe, such as the properties of dark matter and dark energy. Therefore, understanding and characterizing the PSF is essential to properly interpret and extract reliable cosmological information from astronomical observations.

This tutorial (Section 2) produces an image of the PSF model at a specific location in `calexp` and `deepCoadd` images. It then demonstrates how to calculate PSF profiles and contours, along with other properties such as size.

#### Additional Resources

Review article: [Point Spread Function Modelling for Astronomical Telescopes: A Review Focused on Weak Gravitational Lensing Studies](https://ui.adsabs.harvard.edu/abs/2023arXiv230607996L/abstract)

PSF in coadded images:  [Mandelbaum et al. 2022](https://ui.adsabs.harvard.edu/abs/2023OJAp....6E...5M/abstract)

### 1.1. Package Imports

The [matplotlib](https://matplotlib.org/) (and especially sublibrary `matplotlib.pyplot`), [numpy](http://www.numpy.org/), [scipy](https://scipy.org/), and [astropy](http://www.astropy.org/) libraries are widely used Python libraries for plotting, scientific computing, scientific analysis, and astronomical data analysis.

The `lsst.rsp` package provides access to the Table Access Protocol (TAP) service for queries to the DP0 catalogs.

The `lsst.afw.image` provide visualization tools.

The `lsst.afw.display` library provides access to image visualization routines and the `lsst.daf.butler` library is used to access data products via the butler.

The `lsst.afw.analysis.tools` library provides acces to analysis tools.

The `lsst.geom` library provides the representation of a 2D coordinate `Point2D`.

The `lsst.pipe.base` library provides access to the `lsst.pipe.base.Struct` container.

[treecorr](https://rmjarvis.github.io/TreeCorr/_build/html/index.html) by [Jarvis et al. 2004](https://ui.adsabs.harvard.edu/abs/2004MNRAS.352..338J/abstract) is a widely used tool for fast correlations measurements based on a ball tree method (similar to a [k-d tree](https://en.wikipedia.org/wiki/K-d_tree)). In particular, it is widely used in state-of-the-art weak lensing cosmological analyses. 

In [ ]:
%load_ext pycodestyle_magic
%flake8_on
import logging
logging.getLogger("flake8").setLevel(logging.FATAL)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import treecorr
from scipy.optimize import curve_fit
from matplotlib import cm
from numpy.linalg import norm
from matplotlib.ticker import LinearLocator
from matplotlib.offsetbox import AnchoredText

import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.pipe.base as pipeBase
from lsst.rsp import get_tap_service
from lsst.geom import Point2D, radToDeg, SpherePoint, degrees
from lsst.analysis.tools.atools import RhoStatistics

### 1.2 Functions and Parameters Definitions

The following cell will set a standard figure size and `afwDisplay` backend to use throughout the notebook.

In [ ]:
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')
plt.style.use('tableau-colorblind10')

The following hidden cell introduces the conversion factor between the standard deviation of a one-dimensional Gaussian profile and its full-width at half maximum (FWHM), a function characterized by a one-dimensional Gaussian profile, and the class `ImageExaminer`. The `ImageExaminer` class is utilized in section 2.3, "PSF Properties for Rapid Analysis," to scrutinize the image of the Point Spread Function at a specific location.

This class is constructed based on a [similar class](https://github.com/lsst-sitcom/summit_utils/blob/main/python/lsst/summit/utils/imageExaminer.py) developed by Merlin Fisher-Levine. The original class was designed for swift [analysis of images](https://roundtable.lsst.codes/rubintv/summit/auxtel/im_current) captured by the Rubin Auxiliary Telescope using the LATISS instrument, with the results typically displayed on [Rubin TV](https://roundtable.lsst.codes/rubintv). The `ImageExaminer` class includes various methods, such as producing contour, surface, and radial plots, along with calculating specific statistics of interest, including the PSF size (according to specific definitions of size).

Furthermore, the hidden cell incorporates auxiliary functions aimed at plotting instances of the `ImageExaminer` class and computing the properties of a given PSF model at a specific point.

To unveil the content of the hidden cell, opt for "View" from the menu bar, then select "Expand Selected Code." Alternatively, you can click on the vertical line adjacent to the cell or on the three dots that signify the cell's hidden status.

In [ ]:
SIGMATOFWHM = 2.0*np.sqrt(2.0*np.log(2.0))


def gauss(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))


class ImageExaminer():
    """Class to examine PSF image properties.
    This is a simplified version---with minor modifications---of the
    code in
    https://github.com/lsst-sitcom/summit_utils/blob/main/python/lsst/summit/utils/imageExaminer.py
    """
    cutoutMappings = {"fitAmp": "Radial fitted amp",
                      "fitGausMean": "Radial fitted position",
                      "fitFwhm": "Radial fitted FWHM",
                      "eeRadius50": "50% flux radius",
                      "eeRadius80": "80% flux radius",
                      "eeRadius90": "90% flux radius"}

    def __init__(self, exp, *, centroid=None, boxHalfSize=None):
        self.exp = exp
        self.centroid = centroid
        self.data = exp.array
        xlen, ylen = self.data.shape
        if centroid is None:
            self.centroid = np.array([xlen/2, ylen/2])
        else:
            self.centroid = centroid
        if boxHalfSize is None:
            self.boxHalfSize = xlen // 2
        else:
            self.boxHalfSize = boxHalfSize
        self.xx, self.yy = self.getMeshGrid(self.data)
        self.imStats = pipeBase.Struct()
        self.radialAverageAndFit()

    def plotSurface(self, ax=None, useColor=True):
        """Make the surface plot.

        Parameters
        ----------
        ax : `maplotlib.axes`, optional.
            If ``None`` a new figure is created. Supply axes
            if including this as a subplot.
        useColor : `bool`, optional.
            Plot at as a surface if ``True``, else plot as a
            wireframe.
        """
        plotDirect = False
        if not ax:
            fig, ax = plt.subplots(subplot_kw={"projection": "3d"},
                                   figsize=(10, 10))
            plotDirect = True

        if useColor:
            _ = ax.plot_surface(self.xx, self.yy, self.data,
                                cmap=cm.plasma, linewidth=1,
                                antialiased=True, color='k', alpha=0.9)
        else:
            _ = ax.plot_wireframe(self.xx, self.yy,
                                  self.data, cmap=cm.gray,  # noqa F841
                                  linewidth=1, antialiased=True, color='k')

        ax.zaxis.set_major_locator(LinearLocator(10))
        ax.zaxis.set_major_formatter('{x:,.0f}')
        ax.set_title("Surface plot")
        ax.set_xlabel('x (pix)')
        ax.set_ylabel('y (pix)')

        if plotDirect:
            plt.show()

    def plotContours(self, ax=None, nContours=10):
        """Make the contour plot.

        Parameters
        ----------
        ax : `maplotlib.axes`, optional
            If ``None`` a new figure is created. Supply axes
            if including this as a subplot.
        nContours : `int`, optional
            The number of contours to use.
        """
        plotDirect = False
        if not ax:
            fig = plt.figure(figsize=(8, 8))  # noqa F841
            ax = plt.subplot(111)
            plotDirect = True

        vmin = np.percentile(self.data, 0.1)
        vmax = np.percentile(self.data, 99.9)
        lvls = np.linspace(vmin, vmax, nContours)
        _ = ax.contour(self.xx, self.yy,
                       self.data, levels=lvls)

        ax.tick_params(which="both", direction="in",
                       top=True, right=True, labelsize=8)
        ax.set_aspect("equal")
        ax.set_title("Contour plot")
        ax.set_xlabel('x (pix)')
        ax.set_ylabel('y (pix)')

        if plotDirect:
            plt.show()

    def plotRowColSlices(self, ax=None):
        """Make the row and column slice plot.

        Parameters
        ----------
        ax : `maplotlib.axes`, optional
            If ``None`` a new figure is created. Supply axes
            if including this as a subplot.
        """
        rowSlice = self.data[self.boxHalfSize, :]
        colSlice = self.data[:, self.boxHalfSize]

        plotDirect = False
        if not ax:
            ax = plt.subplot(111)
            plotDirect = True

        xs = range(-1*self.boxHalfSize, self.boxHalfSize+1)
        ax.plot(xs, rowSlice, '--', label='Row plot')
        ax.plot(xs, colSlice, label='Column plot')
        ax.set_ylabel('Flux (ADU)')
        ax.set_xlabel('Radius (pix)')
        ax.set_aspect(1.0/ax.get_data_ratio(),
                      adjustable='box')
        ax.legend()
        ax.set_title("PSF slices")
        if plotDirect:
            plt.show()

    def plotRadialAverage(self, ax=None):
        """Make the radial average plot.

        Parameters
        ----------
        ax : `maplotlib.axes`, optional
            If ``None`` a new figure is created. Supply axes
            if including this as a subplot.
        """
        plotDirect = False
        if not ax:
            ax = plt.subplot(111)
            plotDirect = True

        distances = self.radialDistances
        values = self.radialValues
        pars = (self.imStats.fitAmp,
                self.imStats.fitGausMean,
                self.imStats.fitFwhm / SIGMATOFWHM)

        fitFailed = np.isnan(pars).any()

        ax.plot(distances, values, 'x', label='Radial average')
        if not fitFailed:
            fitline = gauss(distances, *pars)
            ax.plot(distances, fitline, label="Gaussian fit")

        ax.set_ylabel('Flux (ADU)')
        ax.set_xlabel('Radius (pix)')
        ax.set_aspect(1.0/ax.get_data_ratio(),
                      adjustable='box')
        ax.legend()
        ax.set_title("Azimuthally-averaged radial profile.")

        if plotDirect:
            plt.show()

    def radialAverageAndFit(self):
        """Calculate flux vs radius from the star's centroid and fit the width.

        Calculate the flux vs distance from the star's
        centroid and fit a Gaussian to get a measurement of the width.

        Also calculates the various encircled energy metrics.

        Notes
        -----
        Nothing is returned, but sets many value in the class.
        """
        xlen, ylen = self.data.shape
        center = np.array([xlen/2, ylen/2])
        distances = []
        values = []

        for i in range(xlen):
            for j in range(ylen):
                value = self.data[i, j]
                dist = norm((i, j) - center)
                if dist > xlen//2:
                    # clip to box size, we don't need a factor
                    # of sqrt(2) extra
                    continue
                values.append(value)
                distances.append(dist)

        peakPos = 0
        amplitude = np.max(values)
        width = 10

        bounds = ((0, 0, 0), (np.inf, np.inf, np.inf))

        try:
            pars, pCov = curve_fit(gauss, distances, values,
                                   [amplitude, peakPos, width],
                                   bounds=bounds)
            pars[0] = np.abs(pars[0])
            pars[2] = np.abs(pars[2])
        except RuntimeError:
            pars = None
            self.imStats.fitAmp = np.nan
            self.imStats.fitGausMean = np.nan
            self.imStats.fitFwhm = np.nan

        if pars is not None:
            self.imStats.fitAmp = pars[0]
            self.imStats.fitGausMean = pars[1]
            self.imStats.fitFwhm = pars[2] * SIGMATOFWHM

        self.radialDistances = distances
        self.radialValues = values

        # calculate encircled energy metric too
        # sort distances and values in step by distance
        d = np.array([(r, v) for (r, v) in sorted(zip(self.radialDistances,
                                                      self.radialValues))])
        self.radii = d[:, 0]
        values = d[:, 1]
        self.cumFluxes = np.cumsum(values)
        self.cumFluxesNorm = self.cumFluxes/np.max(self.cumFluxes)
        self.imStats.eeRadius50 = self.getEncircledEnergyRadius(50)
        self.imStats.eeRadius80 = self.getEncircledEnergyRadius(80)
        self.imStats.eeRadius90 = self.getEncircledEnergyRadius(90)

        return

    def plotCurveOfGrowth(self, ax=None):
        """Make the encircled energy plot.

        Parameters
        ----------
        ax : `maplotlib.axes`, optional
            If ``None`` a new figure is created. Supply axes
            if including this as a subplot.
        """
        plotDirect = False
        if not ax:
            ax = plt.subplot(111)
            plotDirect = True

        ax.plot(self.radii, self.cumFluxesNorm, markersize=10)
        ax.set_ylabel('Encircled flux (%)')
        ax.set_xlabel('Radius (pix)')

        ax.set_aspect(1.0/ax.get_data_ratio(),
                      adjustable='box')
        ax.set_title("Encircled flux")

        if plotDirect:
            plt.show()

    def plotStats(self, ax, lines):
        """Make the stats box 'plot'.

        Parameters
        ----------
        ax : `maplotlib.axes`
            Axes to use.
        lines : `list` of `str`
            The data to include in the text box
        """
        text = "\n".join([line for line in lines])

        stats_text = AnchoredText(text, loc="center", pad=0.5,
                                  prop=dict(size=14, ma="left",
                                            backgroundcolor="white",
                                            color="black", family='monospace'))
        ax.add_artist(stats_text)
        ax.axis('off')

    def getStarBoxData(self):
        """Get the image data for the star.

        Calculates the maximum valid box, and uses that
        to return the image data, setting self.starBbox
        as this method changes the bbox.

        Returns
        -------
        data : `np.array`
            The image data
        """
        bbox = self._calcBbox(self.centroid)
        self.starBbox = bbox

        return self.exp.image[bbox].array

    def getMeshGrid(self, data):
        """Get the meshgrid for a data array.

        Parameters
        ----------
        data : `np.array`
            The image data array.

        Returns
        -------
        xxyy : `tuple` of `np.array`
            The xx, yy as calculated by np.meshgrid
        """
        xlen, ylen = data.shape
        xx = np.arange(-1*xlen/2, xlen/2, 1)
        yy = np.arange(-1*ylen/2, ylen/2, 1)
        xx, yy = np.meshgrid(xx, yy)
        return xx, yy

    def getEncircledEnergyRadius(self, percentage):
        """Radius in pixels with the given percentage of
            encircled energy.

        100% is at the boxHalfWidth dy definition.

        Parameters
        ----------
        percentage : `float` or `int`
            The percentage threshold to return.

        Returns
        -------
        radius : `float`
            The radius at which the ``percentage`` threshold
            is crossed.
        """
        return self.radii[np.argmin(
            np.abs((percentage/100)-self.cumFluxesNorm))]

    @staticmethod
    def translateStats(imStats, mappingDict):
        """Create the text for the stats box from the stats
            themselves.

        Parameters
        ----------
        imStats : `lsst.pipe.base.Struct`
            A container with attributes containing measurements
            and statistics for the image.
        mappingDict : `dict` of `str`
            A mapping from attribute name to name for rendereding
            as text.

        Returns
        -------
        lines : `list` of `str`
            The translated lines of text.
        """
        lines = []
        for k, v in mappingDict.items():
            try:
                value = getattr(imStats, k)
            except Exception:
                lines.append("")
                continue

            if type(value) == float or isinstance(value, np.floating):
                value = f"{value:,.3f}"
            if k == 'centroid':
                value = f"{value[0]:.1f}, {value[1]:.1f}"
            lines.append(f"{v} = {value}")
        return lines


def get_psf_properties(psf, point):
    """Function to obtain PSF properties.

    Parameters
    ----------
    psf : `lsst.meas.extensions.psfex.PsfexPsf`
        PSF object.
    point : `lsst.geom.Point2D`
        Coordinate where the PSF is being evaluated.

    Returns
    -------
        sigma : `float`
            PSF determinat radius from SDDS adaptive
            moments matrix.
        ap_flux : `float`
            PSF flux from aperture photometry weighted
            by a sinc function.
        peak : `float`
            Peak PSF value.
        dims : `lsst.geom.ExtendI`
            PSF postage stamp dimensions.
    """
    sigma = psf.computeShape(point).getDeterminantRadius()
    ap_flux = psf.computeApertureFlux(radius=sigma, position=point)
    peak = psf.computePeak(position=point)
    dims = psf.computeImage(point).getDimensions()

    print(f"PSF size: {sigma:.4} pix \n"
          f"PSF flux from aperture photometry: {ap_flux:.4} \n"
          f"Peak PSF value: {peak:.4} \n"
          f"PSF dimensions: {dims} \n")

    return (sigma, ap_flux, peak, dims)


def plotPsfImageExaminer(psfImageExaminer):
    """Function to plot `ImageExaminer` instances.

    Parameters
    ----------
    psfImageExaminer: `ImageExaminer`
        `ImageExaminer` object.
    """
    figsize = 6
    fig = plt.figure(figsize=(figsize*2, 3*figsize))

    axSurface = fig.add_subplot(321, projection='3d')
    axContour = fig.add_subplot(322)
    axStats = fig.add_subplot(323)
    axSlices = fig.add_subplot(324)
    axRadial = fig.add_subplot(325)
    axEncircled = fig.add_subplot(326)

    psfImageExaminer.plotSurface(axSurface)
    psfImageExaminer.plotContours(axContour)
    psfImageExaminer.plotRowColSlices(axSlices)
    psfImageExaminer.plotRadialAverage(axRadial)

    lines = []
    lines.append("\n     ---- Cutout ----")
    lines.extend(psfImageExaminer.translateStats(
        psfImageExaminer.imStats,
        psfImageExaminer.cutoutMappings))

    psfImageExaminer.plotStats(axStats, lines)
    psfImageExaminer.plotCurveOfGrowth(axEncircled)

    return

### 2. PSF image manipulation in `calexp` and `deepCoadd` images.

The initial stage in processing LSST science observations involves Instrument Signature Removal (ISR). This encompasses fundamental detrending operations like flat-fielding, bias subtraction, fringe correction, and rectification of flawed and oversaturated pixels. Subsequently, the focus shifts to characterizing single-epoch direct images, constructing models that depict the observational system and its conversion of the true celestial scene into the observed image. This process encompasses background subtraction, PSF modeling, addressing cosmic ray effects, applying aperture corrections, and source measurement. After segregating stars and galaxies, the PSF model in single-epoch images is assembled using data from a star catalog. The LSST Science Pipelines have transitioned to using the [PIFF (PSF in the Full-Field of View)](https://rmjarvis.github.io/Piff/_build/html/index.html) code for PSF modeling in the full field of view, replacing the Point Spread Function Extractor, [PSFEx](https://ui.adsabs.harvard.edu/abs/2013ascl.soft01001B/abstract), software. However, the current approach to PSF coaddition is somewhat simplified and not optimally suited for weak lensing research. When the PSF model is requested at a specific point in a coadded image, all the PSF models from the single-epoch images that constitute that coadd are aggregated with the same weighting scheme used for creating the coadded image. The LSST Science Pipelines are in the process of implementing a more robust and clearly defined PSF coaddition procedure (as discussed in [Mandelbaum et al. 2022](https://ui.adsabs.harvard.edu/abs/2023OJAp....6E...5M/abstract)), based on measurements from coadded regions known as "cell-based coadds" (detailed in [Sheldon et al. 2023](https://ui.adsabs.harvard.edu/abs/2023OJAp....6E..17S/abstract)).

In the forthcoming sections, we will generate an image of the PSF model at a specific location within both a `calexp` and a `deepCoadd` image.

#### 2.1 PSF in `calexp` images.

Define the Butler instance for DP0.2 and select a particular `calexp` image. We will pass the `visit` and `detector` numbers in order to uniquely constrain the image, and will visualize the imaga afterwards.

In [ ]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collections)

In [ ]:
datasetType = 'calexp'
dataId = {'visit': 192350, 'detector': 175}
calexp = butler.get(datasetType, dataId=dataId)

The following plot is a 2D image of the `calexp` from the previous butler query, in pixel coordinates ranging from 0 to 4000 pixels in both axes, and with a contrast bar ranging from -300 to 400 digital units. Point and extended sources are scattered around the image. We will retrieve PSF models at particular points in this image.

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(calexp.image)
plt.title(f'Calexp. dataID: {dataId}')
plt.xlabel('x (pix)')
plt.ylabel('y (pix)')
plt.show()

We will use `computeKernelImage` to display the pixel-based model of the PSF at a particular point with coordinates `(x, y) = (2000, 3500)` (denoted by the `point_tuple` variable).

In [ ]:
point_tuple = (2000, 3500)
point_image = Point2D(point_tuple)

In the following cell, we extract the PSF model from the exposure information, obtained from the calexp object itself.

In [ ]:
info_calexp = calexp.getInfo()
psf_calexp = info_calexp.getPsf()

We are now prepared to employ computeKernelImage to evaluate the PSF model at the location defined in the point_tuple variable.

In [ ]:
psf_calexp_kernel = psf_calexp.computeKernelImage(point_image)
first_psf_image_calexp = psf_calexp_kernel.convertF()

The subsequent plot depicts a postage stamp or cut-out of a PSF model extracted from a `calexp` image using the `computeKernelImage` function. The PSF is centered at the origin, and the stamp is rectangular, encompassing 40 pixels and spanning from negative 20 to positive 20 pixels on each side. The color contrast bar spans a range from approximately negative 0.0002 to about 0.0003.

In [ ]:
fig = plt.figure(figsize=(10, 8))
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(first_psf_image_calexp)
plt.title(f'PSF model in calexp at {point_image} using computeKernelImage')
plt.xlabel('x (pix)')
plt.ylabel('y (pix)')
plt.gca().axis('on')

Note that the image coordinates are centered at the origin of the image. The coordinates of this origin point are (0,0), resulting in negative coordinates for the lower left point:

In [ ]:
print(first_psf_image_calexp.getXY0())

Now, instead of using `computeKernelImage`, we will utilize `computeImage`. The former positions the PSF center at the center of the central pixel within the stamp or cutout (`computeKernelImage`). On the other hand, the latter (`computeImage`) allows the PSF center to be placed at any arbitrary location. To achieve this, we require the astrometric solution, often referred to as the "World Coordinate System" (WCS) in the code, which maps pixel coordinates to sky coordinates and can also be obtained form the exposure information, as the PSF model. The "WCS" nomenclature is used for historical reasons. Note also that we need to convert the `point_tuple` variable into an `lsst.geom.Point2D` object first. 

In [ ]:
wcs_calexp = info_calexp.getWcs()
point_object = Point2D(point_tuple)
second_psf_image_calexp = psf_calexp.computeImage(point_object).convertF()

The following plot shows a postage stamp or cut-out of a PSF model from a `calexp`, obtained using `computeKernelImage`. The PSF is centered around (2000, 3500), and the stamp is rectangular, with a size of 40 pixels, ranging from 1980 to 2020 pixels on the `x` axis and from about 3480 to 3520 on the `y` axis. The contrast bar ranges from negative 0.0002 to about 0.0003. 

In [ ]:
fig = plt.figure(figsize=(10, 8))
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(second_psf_image_calexp)
plt.title(f'PSF model in calexp at {point_image} using computeImage')
plt.xlabel('x (pix)')
plt.ylabel('y (pix)')
plt.gca().axis('on')

Check again the coordinates for the lower left point in the previous image:

In [ ]:
print(second_psf_image_calexp.getXY0())

Use now one of the helper functions we defined earlier — `get_psf_properties` — to calculate some PSF properties at a particular point:

In [ ]:
props = get_psf_properties(psf_calexp, point_image)

#### 2.2 PSF in `deepCoadd` images.

When conducting multi-epoch surveys for static-sky science, the traditional method involves creating coadds. This process entails resampling images from different observations onto a common grid and averaging them to generate a single, deeper image known as a `deepCoadd`. Additionally, a coadded point spread function (PSF) model is established. Handling PSF coadding with care is crucial to ensure a well-defined PSF.

One of the challenges in coadding PSFs arises from slight variations observed between PSFs in different visits. Even minor differences make it practically impossible to accurately model the effective PSF of the coadd using coadded star images. Even small positional shifts, known as dithers, used to fill the gaps between charge-coupled devices introduce disruptions in the effective PSF of the coadd. As the number of dithers increases, the areas within the coadd with a continuous effective PSF become smaller, making it increasingly unlikely to find stars suitable for PSF modeling in each region. For the Hyper-Suprime Camera survey, [Bosch et al 2018](https://ui.adsabs.harvard.edu/abs/2018PASJ...70S...5B/abstract) use an approach that involves resampling and combining existing PSF models from the input images using the same coordinate transformations and weights applied to the image data. The Hyper-Suprime Camera survey analysis pipeline is based on the LSST Science Pipelines code.

[Mandelbaum et al. 2022](https://ui.adsabs.harvard.edu/abs/2023OJAp....6E...5M/abstract) provide a mathematical framework that aids in producing well-defined coadded PSFs. It's important to note that the results obtained using this approach are specifically applicable to static sources. As indicated in footnote 2 on page 3 of [Mandelbaum et al. 2022](https://ui.adsabs.harvard.edu/abs/2023OJAp....6E...5M/abstract), the coadd PSF for time-varying sources will vary depending on their light curves. For instance, if a variable or transient source appears very bright in just one exposure and faint in all others, its coadd PSF will essentially match that of the exposure where it is bright. This effect may introduce additional challenges for the LSST. The initial survey plan proposed using two 15-second "snaps" combined into a single 30-second exposure for each visit. Whether this approach will be implemented is still under consideration, but if adopted, the issue of PSF for time-varying objects in a coadd becomes relevant even in the process of combining the two snaps.

Visualize first the PSF model in a `deepCoadd` image similar to section 2.1, but this time using a `deepCoadd` image. 

Select the same point in the sky that we used to represent the PSF in the `calexp` of the previous section. 

This will allow us to better appreciate the differences between the `deepCoadd` and an individual `calexp` PSF model.

First, retrieve the right ascension and declination coordinates of the point we used in the previous section (stored in the variable `point_tuple`), as well as the band of that exposure. Then, we will follow the steps outlined in section 3.2 of the Notebook tutorial `DP02_01` to determine the `tract` and `patch` of the `deepCoadd`. 

This information, along with the `band`, will uniquely define a data ID that we can provide to the butler to access the corresponding `deepCoadd`.

This time, we will pass both the `tract` and `patch` numbers, along with the desired band (`i`), to uniquely specify the image we're interested in.

In [ ]:
x, y = point_tuple
ra, dec = wcs_calexp.pixelToSky(x, y)
my_ra_deg = radToDeg(ra)
my_dec_deg = radToDeg(dec)
print(my_ra_deg, my_dec_deg)

In [ ]:
my_band = info_calexp.getFilter().bandLabel
print(my_band)

In [ ]:
my_spherePoint = SpherePoint(my_ra_deg*degrees,
                             my_dec_deg*degrees)
print(my_spherePoint)

skymap = butler.get('skyMap')

tract = skymap.findTract(my_spherePoint)
patch = tract.findPatch(my_spherePoint)

my_tract = tract.tract_id
my_patch = patch.getSequentialIndex()

print('my_tract: ', my_tract)
print('my_patch: ', my_patch)

In [ ]:
datasetType = 'deepCoadd'
dataId = {'tract': my_tract, 'patch': my_patch, 'band': my_band}
coadd = butler.get(datasetType, dataId=dataId)

The following plot is a 2D image of the `deepCoadd` from the previous butler query, in pixel coordinates ranging from 12000 to 16000 pixels in the horizontal axis and 0 to 4000 pixels in the vertical axis. The contrast bar ranges from negative 0.2 to 0.3 digital units. Point and extended sources are scattered around the image, however, a galaxy cluster is particularly prominent inthe lower left part of the image.

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(coadd.image)
plt.title(f'deepCoadd. dataID: {dataId}')
plt.xlabel('x (pix)')
plt.ylabel('y (pix)')
plt.show()

In order to use the same point as in the example above with the `calexp`, we will retrieve first the WCS for the coadd information, and convert the sky point to local pixel coordinates in the `deepCoadd` image displayed in the previous cell.   

In [ ]:
info_coadd = coadd.getInfo()
wcs_coadd = info_coadd.getWcs()
point_image = wcs_coadd.skyToPixel(my_spherePoint)
point_image

Use `computeKernelImage` to display the pixel-based model of the PSF at a particular point. Note the higher signal-to-noise ratio in this image, compared to the single `calexp` PSF image in Section 2.1

In [ ]:
psf_coadd = info_coadd.getPsf()
psf_kernel_coadd = psf_coadd.computeKernelImage(point_image)
first_psf_image_coadd = psf_kernel_coadd.convertF()

The following plot shows a postage stamp or cut-out of a PSF model from a `deepCoadd` image, obtained using `computeKernelImage`. The PSF is centered around zero, and the stamp is rectangular, with a size of about 50 pixels, ranging from -25 to 25 pixels on each size. The contrast bar ranges from -1e-5 to 7e-5 . 

In [ ]:
fig = plt.figure(figsize=(10, 8))
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(first_psf_image_coadd)
plt.title(f'PSF model in deepCoadd at {point_image} using computeKernelImage')
plt.xlabel('x (pix)')
plt.ylabel('y (pix)')
plt.gca().axis('on')

Print the coordinates of the lower-left corner:

In [ ]:
print(first_psf_image_coadd.getXY0())

Now, instead of `computeKernelImage`, use
`computeImage`. 

One has the PSF center at the center of the 
central pixel in the stamp or cutout (computeKernelImage), and 
the other one (computeImage) can make it at any arbitrary location.

In [ ]:
second_psf_image_coadd = psf_coadd.computeImage(point_image).convertF()

The following plot shows a postage stamp or cut-out of a PSF model from a `deepCoadd` image, obtained using `computeImage`. The PSF is no longer centered around zero as when we used `computeKernelImage`, but centered at about 13000 and 2325 pixels. The stamp is still rectangular, with a size of about 50 pixels, ranging from 12970 to 12970 pixels on the horizantal axis and 2250 to 2350 on the vertical axis. The contrast bar ranges from negative 1e-5 to 7e-5 .

In [ ]:
fig = plt.figure(figsize=(10, 8))
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
afw_display.mtv(second_psf_image_coadd)
plt.title(f'PSF model in deepCoadd at {point_image} using computeImage')
plt.xlabel('x (pix)')
plt.ylabel('y (pix)')
plt.gca().axis('on')
plt.show()

Look at the coordinates of the lower-left corner:

In [ ]:
second_psf_image_coadd.getXY0()

Calculate the PSF properties, as in Section 2.1 for the `calexp` PSF:

In [ ]:
props_psf_coadd = get_psf_properties(psf_coadd, point_image)

#### 2.3.  PSF properties for rapid analysis.

In this section, we will utilize the `ImageExaminer` class that was introduced in section 1.2 of this tutorial. Our aim is to visualize the PSFs from both `calexp` and `deepCoadd` images discussed in sections 2.1 and 2.2. We will achieve this by employing surface and contour plots. Additionally, we will generate one-dimensional profiles of the central PSF along both the horizontal and vertical axes, along with creating an azimuthally-averaged radial profile of the PSF. We intend to fit a Gaussian function to this radial profile. Lastly, we will present a plot depicting the encircled flux as a function of radius. We will also showcase the results of fitting a Gaussian function to the radial profile, including parameters such as amplitude, size (Full-Width at Half-Maximum), and centroid, along with the radii corresponding to 50%, 80%, and 90% encircled fluxes.

The code for this section is grounded in the [imageExaminer.py](https://github.com/lsst-sitcom/summit_utils/blob/main/python/lsst/summit/utils/imageExaminer.py) class. This class is currently employed for the [swift PSF analysis of the LATISS images](https://roundtable.lsst.codes/rubintv/summit/auxtel/im_current) obtained by Rubin's AuxTel at Cerro Pachón, Chile.

Let's now produce the plots for the`calexp` PSF of section 2.1. We'll start by making an instance of the `ImageExaminer` class.

In [ ]:
psfImageExaminerCalexp = ImageExaminer(first_psf_image_calexp)

The following cell will generate a plot comprising 6 panels, organized into 3 rows and 2 columns. In the upper left plot, a three-dimensional surface representation of the `calexp` PSF within the specified postage stamp range will be displayed. The upper right plot will present a two-dimensional contour plot showcasing the PSF centered at its core.

Moving to the middle row, the left panel will provide a condensed overview of key statistics. These statistics will report the amplitude, width, and centroid derived from a one-dimensional Gaussian fit applied to the azimuthally averaged radial profile of the PSF. Furthermore, the panel will detail three encircled flux radii, corresponding to 50%, 80%, and 90% of the encircled flux, respectively. The right panel of this middle row will display the one-dimensional radial profiles. These profiles are formed by slicing the 2D PSF image along the central axes in both the horizontal and vertical directions.

The left panel of the final row will depict the azimuthally averaged PSF radial profile, accompanied by a curve representing the Gaussian fit to the data. The right panel of this row will showcase a cumulative plot with a monotonically increasing curve, illustrating the percentage of encircled flux emanating from the PSF profile's center as a function of the radius in pixels.

In [ ]:
plotPsfImageExaminer(psfImageExaminerCalexp)

And now let's do the plotting for the PSF from the `deppCoadd` of section 2.2.

In [ ]:
psfImageExaminerCoadd = ImageExaminer(first_psf_image_coadd)

The upcoming cell will generate a plot featuring 6 panels, arranged in a grid with 3 rows and 2 columns. In the upper left plot, a three-dimensional surface representation of the `deepCoadd` PSF within the specified postage stamp range will be depicted. The upper right plot will display a two-dimensional contour representation of the PSF, centered at its core.

Moving to the middle row, the left panel will encapsulate a summary of crucial statistics. These statistics encompass the amplitude, width, and centroid derived from a one-dimensional Gaussian fit applied to the azimuthally averaged radial profile of the PSF. Additionally, this panel will provide details about three encircled flux radii, corresponding to 50%, 80%, and 90% of the encircled flux, respectively. The right panel of this middle row will exhibit the one-dimensional radial profiles, constructed by slicing the 2D PSF image along the central axes in both horizontal and vertical directions.

The left panel of the final row will visualize the azimuthally averaged PSF radial profile, alongside a curve resulting from the Gaussian fit to the data. The right panel of this row will showcase a cumulative plot with a monotonically increasing curve, illustrating the percentage of encircled flux originating from the center of the PSF profile as a function of radius in pixels. It's noteworthy that since the displayed PSF is a model built from coadded images, the plots and profiles exhibit a higher signal-to-noise ratio in comparison to the analogous plots from a previous cell, created for a `calexp` PSF.

In [ ]:
plotPsfImageExaminer(psfImageExaminerCoadd)

### Exercises for the Learner

#### Section 2:

* a) In the `radialAverageAndFit` function of the `ImageExaminer` class, try using a different functional form for fitting the one-dimensional PSF radial profiles, such as the Moffat profile (as described in section 3.1 of [Jarvis et al. 2021](https://ui.adsabs.harvard.edu/abs/2021MNRAS.501.1282J/abstract)).

* b) Extend the analysis of the one-dimensional PSF radial profiles by calculating additional statistics such as skewness and kurtosis, and visualize them to gain insights into the PSF behavior.